In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, explode
import os
import re

In [3]:
# Se crea la sesión de Spark
conf = SparkConf().set("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem")
spark = SparkSession.builder \
        .config(conf=conf) \
        .appName('Clima Procesamiento') \
        .getOrCreate()

In [4]:
def obtener_ultimo_archivo(directorio, extension = '*.json'):
    
    '''
    Obtiene el archivo más reciente de un directorio con la extensión especificada.

    Args:
        directorio (str): Ruta del directorio donde buscar los archivos.
        extension (str): Extensión de los archivos a buscar (por defecto '*.json').

    Returns:
        str: Ruta completa del archivo más reciente con la extensión especificada.
        '''
    
    try:
        # Obtener la lista de rutas completas de los archivos que coinciden con la extensión
        archivos = [os.path.join(directorio, archivo) for archivo in os.listdir(directorio) if archivo.endswith(extension)]
        
        # Verificar si no hay archivos en la lista
        if not archivos:
            raise FileNotFoundError('No se encontraron archivos en el directorio especificado.')
        
        # Obtener el archivo más reciente basado en la fecha de modificación
        ultimo_archivo = max(archivos, key = os.path.getmtime)
        
        # Retorna ruta completa de último archivo
        return ultimo_archivo
    
    except FileNotFoundError as e:
        print(f'Error: {e}')
        raise
    except Exception as e:
        print(f'Error inesperado: {e}')
        raise

data_dir = 'Datos'

ultimo_archivo = obtener_ultimo_archivo(data_dir, extension='.json')

In [206]:
df = spark.read.json(ultimo_archivo)
df.show()


+-------+------+--------------------+--------------------+--------------------+--------+---------+---------+---------------+--------------------+-----------+--------+
|address|alerts|   currentConditions|                days|         description|latitude|longitude|queryCost|resolvedAddress|            stations|   timezone|tzoffset|
+-------+------+--------------------+--------------------+--------------------+--------+---------+---------+---------------+--------------------+-----------+--------+
|Sicilia|    []|{88.0, Partially ...|[{62.5, Rain, Par...|Similar temperatu...| 38.1221|  13.3611|        1|Sicilia, Italia|{{0.0, 54398.0, C...|Europe/Rome|     1.0|
+-------+------+--------------------+--------------------+--------------------+--------+---------+---------+---------------+--------------------+-----------+--------+



In [212]:
for key, df in dfDesanidadoStruct_DaysHours_4.items():
    print(f'- LLAVE DF: {key}')
    df.show()

- LLAVE DF: days_hours
+---------------------+---------------------+-------------------+------------------------+--------------+--------------------+-------------------+-------------------+-----------------+---------------------+---------------------+-------------------+---------------------+---------------+--------------------+----------------------+-------------------------+-----------------+-------------------+---------------+------------------+---------------------+------------------+-------------------+--------------------+
|days_hours_cloudcover|days_hours_conditions|days_hours_datetime|days_hours_datetimeEpoch|days_hours_dew|days_hours_feelslike|days_hours_humidity|    days_hours_icon|days_hours_precip|days_hours_precipprob|days_hours_preciptype|days_hours_pressure|days_hours_severerisk|days_hours_snow|days_hours_snowdepth|days_hours_solarenergy|days_hours_solarradiation|days_hours_source|days_hours_stations|days_hours_temp|days_hours_uvindex|days_hours_visibility|days_hours_win

### PROCESAMIENTO DE DATOS 

In [217]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import array, col, concat_ws
from pyspark.sql.types import StructType, ArrayType, StringType, DoubleType, LongType, FloatType
from pyspark.sql.types import StringType, IntegerType, LongType, DoubleType


def explotar_columnas_array(df, diccionario_resultado, sufijo_explode=None, columnas_target=None):
    
    for columna in df.schema:
        if isinstance(columna.dataType, ArrayType):
            columna_nombre = columna.name
            if columnas_target is None or columna_nombre in columnas_target:
                diccionario_resultado[columna_nombre] = df.select(explode(col(columna_nombre)).alias(f'{columna_nombre}'))
            
def desanidar_columnas_struct(df, diccionario_resultado, sufijo_desanidado=None, columnas_target=None):
    
    for columna in df.schema:
        if isinstance(columna.dataType, StructType):
            columna_nombre = columna.name
            if columnas_target is None or columna_nombre in columnas_target:

                campos_struct = [
                    col(f'{columna_nombre}.{subfield.name}').alias(f'{columna_nombre}_{subfield.name}')
                    for subfield in columna.dataType.fields
                ]
                diccionario_resultado[columna_nombre] = df.select(*campos_struct)
                    
def aplicar_dataframe(metodo:str, diccionario_df, diccionario_dfResultado, sufijo=None, columnas_target=None):
    
    if metodo == 'explotar':
        if isinstance(diccionario_df, DataFrame):
            explotar_columnas_array(diccionario_df, diccionario_dfResultado, sufijo, columnas_target)
        
        elif isinstance(diccionario_df, dict):
            for key, df in diccionario_df.items():
                explotar_columnas_array(df, diccionario_dfResultado, sufijo, columnas_target)
    
    elif metodo == 'desanidar':
        if isinstance(diccionario_df, DataFrame):
            desanidar_columnas_struct(diccionario_df, diccionario_dfResultado, sufijo, columnas_target)
        
        elif isinstance(diccionario_df, dict):
            for key, df in diccionario_df.items():
                desanidar_columnas_struct(df, diccionario_dfResultado, sufijo, columnas_target)
    
def transformar_dfPandas(diccionario_df, nombre_dataframe=None):
    
    try:    
        if nombre_dataframe:
            nombre_df = diccionario_df[nombre_dataframe]
            dfPandas = nombre_df.toPandas()
            
            print(f'DataFrame de Spark del diccionario "{nombre_dataframe}" convertido a DataFrame de Pandas.')
            return dfPandas

        else:
            dfPandas = diccionario_df.toPandas()
            
            return dfPandas
            print('DataFrame de Spark convertido a DataFrame de Pandas.')
            
    except Exception as e:
        print(f'Error en la conversión de DF de Spark a DF de Pandas: {e}')
                
def unificar_df(diccionario_df):

    try:
            df_list = []  # Lista para almacenar DataFrames normalizados

            for key, df in diccionario_df.items():
                # Obtener nombres de columnas
                columnas_originales = df.schema.names

                # Extraer el prefijo (por ejemplo, "stations_C6242_")
                prefijo = key + "_"  # Usa la clave del diccionario como prefijo

                # Verificar si las columnas realmente tienen el prefijo antes de renombrarlas
                df_renombrado = df.select(
                    [col(c).alias(c.replace(prefijo, "")) if c.startswith(prefijo) else col(c) for c in columnas_originales]
                )

                df_list.append(df_renombrado)  # Agregar el DF transformado

            # Unificar todos los DataFrames en uno solo
            if df_list:
                df_final = df_list[0]
                for df in df_list[1:]:
                    df_final = df_final.unionByName(df, allowMissingColumns=True)  # Une permitiendo columnas faltantes
                
                print('DataFrame unificado.')
                return df_final
        
    except Exception as e:
        print(f"❌ No se pudo unificar el DataFrame, revisa los datos de entrada: {e}")
     
def reemplazar_nulos(diccionario_df):
    
    # REEMPLAZO VALORES NULOS
    valores_reemplazo = { 
    StringType : 'Sin Dato',
    IntegerType : 0,
    LongType : 0,
    DoubleType : 0.0}   
    
    if isinstance(diccionario_df, dict):
        
        for key, df in diccionario_df.items():
            # Reemplazar según tipo de dato
            for columna in df.schema.fields:
                tipo = columna.dataType
                
                if isinstance(tipo, ArrayType) and isinstance(tipo.elementType, StringType):
                    df = df.withColumn(
                        columna.name,
                        when(col(columna.name).isNull(), array(lit('Sin Dato')))
                        .otherwise(col(columna.name))
                    )
                
                elif type(tipo) in valores_reemplazo:
                    df = df.fillna({columna.name : valores_reemplazo[type(tipo)]})
            
            diccionario_df[key] = df
              
def eliminar_columna(diccionario_df, nombre_columna, nombre_dataframe=None):
    
    try:
        if nombre_dataframe:
            df = diccionario_df[nombre_dataframe]
            df_eliminacionColumna = df.drop(*nombre_columna)
            
            print(f'Columna {nombre_columna} eliminada del DataFrame {nombre_dataframe}.')
            return df_eliminacionColumna
        
        else:
            df_eliminacionColumna = diccionario_df.drop(*nombre_columna)
            
            print(f'Eliminación correcta de columnas {nombre_columna} del DataFrame.')
            return df_eliminacionColumna
            
    
    except Exception as e:
        print(f'Error en la eliminación de columna "{nombre_columna}" del DataFrame "{nombre_dataframe}: {e}"')
        

def eliminar_corchetes_array(diccionario_df, nombre_dataframe, columna_nueva, columna_original):
    
    if isinstance(columna_nueva, list) and isinstance(columna_original, list):
    df = diccionario_df[nombre_dataframe]
    
    for nuevo, orig in zip(columna_nueva, columna_original):
        df = df.withColumn(nuevo, concat_ws(', ', orig)).drop(orig)
    
    return df


def guardar_csv(df, ruta_csv):
    
    try:
        df.to_csv(ruta_csv)
    
    except Exception as e:
        print(f'Error en cargar el DataFrame de Pandas en formato CSV: {e}')
    
    
# 1 -TERMINAR DE CONFIGURAR FUNCIÓN "eliminar_corchetes_array" 
#   1.1 - DEBE PODER RECIBIR LISTAS DE COLUMNAS Y COLUMNAS INDIVIDUALES   
        
    
    

In [218]:
eliminar_corchetes = eliminar_corchetes_array(dfDesanidadoStruct_Current_Station_1, 'currentConditions', 'stations', 'currentConditions_stations')
#eliminar_corchetes2 = eliminar_corchetes_array(dfDesanidadoStruct_DaysHours_4, 'days_hours', ['daysHours_preciptype', 'daysHours_stations'], ['days_hours_preciptype', 'days_hours_stations'])

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `c` cannot be resolved. Did you mean one of the following? [`currentConditions_dew`, `currentConditions_icon`, `currentConditions_snow`, `currentConditions_temp`, `currentConditions_precip`].;
'Project [currentConditions_cloudcover#88306, currentConditions_conditions#88365, currentConditions_datetime#88424, currentConditions_datetimeEpoch#88483L, currentConditions_dew#88542, currentConditions_feelslike#88601, currentConditions_humidity#88660, currentConditions_icon#88719, currentConditions_moonphase#88778, currentConditions_precip#88837, currentConditions_precipprob#88896, currentConditions_preciptype#88955, currentConditions_pressure#89014, currentConditions_snow#89073, currentConditions_snowdepth#89132, currentConditions_solarenergy#89191, currentConditions_solarradiation#89250, currentConditions_source#89309, currentConditions_stations#89339, currentConditions_sunrise#89398, currentCondit

In [216]:
eliminar_corchetes2.show()

+---------------------+---------------------+-------------------+------------------------+--------------+--------------------+-------------------+-------------------+-----------------+---------------------+-------------------+---------------------+---------------+--------------------+----------------------+-------------------------+-----------------+---------------+------------------+---------------------+------------------+-------------------+--------------------+--------------------+------------------+
|days_hours_cloudcover|days_hours_conditions|days_hours_datetime|days_hours_datetimeEpoch|days_hours_dew|days_hours_feelslike|days_hours_humidity|    days_hours_icon|days_hours_precip|days_hours_precipprob|days_hours_pressure|days_hours_severerisk|days_hours_snow|days_hours_snowdepth|days_hours_solarenergy|days_hours_solarradiation|days_hours_source|days_hours_temp|days_hours_uvindex|days_hours_visibility|days_hours_winddir|days_hours_windgust|days_hours_windspeed|daysHours_preciptype|

In [207]:

dfExplodedArray_Alerts_Days_1 = {}
dfDesanidadoStruct_Days_2 = {}
dfExplodeArray_DaysHours_DayStation_3 = {} 
dfDesanidadoStruct_DaysHours_4 = {} 

dfDesanidadoStruct_Current_Station_1 = {}

dfDesanidadoStruct_Stations_2 = {}


columnas_array_1 = ['alerts', 'days']
columnas_struct_1 = ['currentConditions', 'stations']

columnas_struct_2 = ['days']
columnas_array_2 = ['days_hours', 'days_stations']

columnas_struct_3 = ['days_hours']

columnas_struct_4 = {'stations_C6242', 'stations_D2770', 'stations_LICJ', 'stations_LICT'}


aplicar_dataframe('explotar', df, dfExplodedArray_Alerts_Days_1, 'explode1', columnas_array_1)
aplicar_dataframe('desanidar', df, dfDesanidadoStruct_Current_Station_1, 'desanidar1', columnas_struct_1)
aplicar_dataframe('desanidar', dfExplodedArray_Alerts_Days_1, dfDesanidadoStruct_Days_2, 'desanidar2', columnas_struct_2)
aplicar_dataframe('explotar', dfDesanidadoStruct_Days_2, dfExplodeArray_DaysHours_DayStation_3, 'desanidar2', columnas_array_2)
aplicar_dataframe('desanidar', dfExplodeArray_DaysHours_DayStation_3, dfDesanidadoStruct_DaysHours_4, 'desanidar2', columnas_struct_3)
aplicar_dataframe('desanidar', dfDesanidadoStruct_Current_Station_1, dfDesanidadoStruct_Stations_2, 'desanidar2', columnas_struct_4)


reemplazar_nulos(dfExplodedArray_Alerts_Days_1)
reemplazar_nulos(dfDesanidadoStruct_Days_2)
reemplazar_nulos(dfExplodeArray_DaysHours_DayStation_3)
reemplazar_nulos(dfDesanidadoStruct_DaysHours_4)
reemplazar_nulos(dfDesanidadoStruct_Current_Station_1)
reemplazar_nulos(dfDesanidadoStruct_Stations_2)

# ELIMINACIÓN DE COLUMNAS STRUCT/ARRAY
df_Days_2_eliminacionColumna = eliminar_columna(dfDesanidadoStruct_Days_2, ['days_hours', 'days_stations'], 'days')
df_currentConditions_eliminacionColumna = eliminar_columna(dfDesanidadoStruct_Current_Station_1, 'currentConditions_stations', 'currentConditions')
df_original = eliminar_columna(df, ['alerts', 'currentConditions', 'days', 'stations'])

# UNIFICACIONES
dfUnificado_stations = unificar_df(dfDesanidadoStruct_Stations_2)

# TRANSFORMACIONES A PANDAS
dfPandas_stations = transformar_dfPandas(dfUnificado_stations)
dfPandas_currentConditions = transformar_dfPandas(dfDesanidadoStruct_Current_Station_1, 'currentConditions')
dfPandas_Days = transformar_dfPandas(df_Days_2_eliminacionColumna)
dfPandas_DayHours = transformar_dfPandas(dfDesanidadoStruct_DaysHours_4, 'days_hours')
dfPandas_Original = transformar_dfPandas(df_original)

# GUARDAR DF DE PANDAS EN FORMATO CSV
guardar_csv(dfPandas_stations, 'Datos/Datos_Procesados/Stations.csv')
guardar_csv(dfPandas_currentConditions, 'Datos/Datos_Procesados/CurrentConditions.csv')
guardar_csv(dfPandas_Days, 'Datos/Datos_Procesados/Days.csv')
guardar_csv(dfPandas_DayHours, 'Datos/Datos_Procesados/DaysHours.csv')
guardar_csv(dfPandas_Original, 'Datos/Datos_Procesados/Original.csv')

Columna ['days_hours', 'days_stations'] eliminada del DataFrame days.
Columna currentConditions_stations eliminada del DataFrame currentConditions.
Eliminación correcta de columnas ['alerts', 'currentConditions', 'days', 'stations'] del DataFrame.
DataFrame unificado.
DataFrame de Spark del diccionario "currentConditions" convertido a DataFrame de Pandas.
DataFrame de Spark del diccionario "days_hours" convertido a DataFrame de Pandas.


In [183]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Muestra todas las columnas

dfPandas_currentConditions.head()

,currentConditions_cloudcover,currentConditions_conditions,currentConditions_datetime,currentConditions_datetimeEpoch,currentConditions_dew,currentConditions_feelslike,currentConditions_humidity,currentConditions_icon,currentConditions_moonphase,currentConditions_precip,currentConditions_precipprob,currentConditions_preciptype,currentConditions_pressure,currentConditions_snow,currentConditions_snowdepth,currentConditions_solarenergy,currentConditions_solarradiation,currentConditions_source,currentConditions_stations,currentConditions_sunrise,currentConditions_sunriseEpoch,currentConditions_sunset,currentConditions_sunsetEpoch,currentConditions_temp,currentConditions_uvindex,currentConditions_visibility,currentConditions_winddir,currentConditions_windgust,currentConditions_windspeed
0,88.0,Partially cloudy,13:50:00,1738155000,47.2,56.1,72.0,partly-cloudy-day,0.0,0.0,0.0,Sin Dato,1016.0,0.0,0.0,0.4,115.0,obs,"[C6242, LICJ, D2770]",07:13:44,1738131224,17:26:13,1738167973,56.1,1.0,6.2,215.0,7.5,8.5


In [ ]:
dfDesanidadoStruct_Current_Station_1['currentConditions'].printSchema().show()

In [186]:
dfPandas_DayHours.head()

,days_hours_cloudcover,days_hours_conditions,days_hours_datetime,days_hours_datetimeEpoch,days_hours_dew,days_hours_feelslike,days_hours_humidity,days_hours_icon,days_hours_precip,days_hours_precipprob,days_hours_preciptype,days_hours_pressure,days_hours_severerisk,days_hours_snow,days_hours_snowdepth,days_hours_solarenergy,days_hours_solarradiation,days_hours_source,days_hours_stations,days_hours_temp,days_hours_uvindex,days_hours_visibility,days_hours_winddir,days_hours_windgust,days_hours_windspeed
0,25.0,Partially cloudy,00:00:00,1738105200,49.8,54.9,82.89,partly-cloudy-night,0.0,0.0,[Sin Dato],1017.0,10.0,0.0,0.0,0.0,0.0,obs,"[C6242, LICJ, LICT]",54.9,0.0,6.2,196.0,12.3,4.7
1,25.0,Partially cloudy,01:00:00,1738108800,48.3,55.0,77.95,partly-cloudy-night,0.0,0.0,[rain],1016.2,10.0,0.0,0.0,0.0,0.0,obs,"[C6242, LICJ, LICT]",55.0,0.0,6.2,210.0,7.8,5.7
2,25.0,Partially cloudy,02:00:00,1738112400,46.8,56.8,69.03,partly-cloudy-night,0.0,0.0,[Sin Dato],1016.1,10.0,0.0,0.0,0.0,0.0,obs,"[C6242, LICJ, LICT]",56.8,0.0,6.2,172.0,13.0,11.0
3,50.0,Partially cloudy,03:00:00,1738116000,36.9,58.5,44.41,partly-cloudy-night,0.0,0.0,[rain],1015.2,10.0,0.0,0.0,0.0,0.0,obs,"[C6242, LICJ, LICT]",58.5,0.0,6.2,189.0,11.6,10.8
4,50.0,Partially cloudy,04:00:00,1738119600,46.8,56.8,69.03,partly-cloudy-night,0.0,0.0,[rain],1016.1,10.0,0.0,0.0,0.0,0.0,obs,"[C6242, LICJ, LICT]",56.8,0.0,6.2,228.0,13.4,19.3


## CARGA DE ARCHIVOS A MYSQL

In [200]:
import pymysql
def insertar_datos_a_tabla(cursor, tabla, columnas, datos):
    
    try:
        placeholders = ', '.join(['%s'] * len(columnas))
        insert_query = f"INSERT INTO {tabla} ({', '.join(columnas)}) VALUES ({placeholders})"

        cursor.executemany(insert_query, datos)
        print(f'Datos insertados correctamente en la tabla {tabla}.')

    except Exception as e:
        print(f'Error en la inserción de datos: "{tabla}" / {e}')


def crear_tablas(cursor):
    
    try:
        tablas = [
            ''' 
            CREATE TABLE IF NOT EXISTS stations (
               id_stations INT AUTO_INCREMENT PRIMARY KEY,
               contribution FLOAT,
               distance FLOAT,
               id VARCHAR(15),
               latitude FLOAT,
               longitude FLOAT,
               name VARCHAR(50),
               quality INT,
               useCount INT
            );
            ''',
            '''
            CREATE TABLE IF NOT EXISTS current_conditions (
            id_currentConditions INT AUTO_INCREMENT PRIMARY KEY,
            currentConditions_cloudcover FLOAT,
            currentConditions_conditions VARCHAR(250),
            currentConditions_datetime TIME,
            currentConditions_datetimeEpoch INT, 
            currentConditions_dew FLOAT,
            currentConditions_feelslike FLOAT,
            currentConditions_humidity FLOAT,
            currentConditions_icon VARCHAR(100),
            currentConditions_moonphase FLOAT,
            currentConditions_precip FLOAT,
            currentConditions_precipprob FLOAT,
            currentConditions_preciptype VARCHAR(100),
            currentConditions_pressure FLOAT,
            currentConditions_snow FLOAT, 
            currentConditions_snowdepth FLOAT,
            currentConditions_solarenergy FLOAT,
            currentConditions_solarradiation FLOAT,
            currentConditions_source VARCHAR(50),
            currentConditions_stations VARCHAR(100),
            currentConditions_sunrise TIME,
            currentConditions_sunriseEpoch INT, 
            currentConditions_sunset TIME,
            currentConditions_sunsetEpoch INT,
            currentConditions_temp FLOAT,
            currentConditions_uvindex FLOAT, 
            currentConditions_visibility FLOAT,
            currentConditions_winddir FLOAT,
            currentConditions_windgust FLOAT, 
            currentConditions_windspeed FLOAT
            );
            ''',
            ''' 
            CREATE TABLE IF NOT EXISTS days (
                id_days INT AUTO_INCREMENT PRIMARY KEY,
                days_cloudcover FLOAT,
                days_conditions VARCHAR(200),
                days_datetime DATE,
                days_datetimeEpoch INT,
                days_description VARCHAR(200),
                days_dew FLOAT,
                days_feelslike FLOAT,
                days_feelslikemax FLOAT, 
                days_feelslikemin FLOAT, 
                days_humidity FLOAT, 
                days_icon VARCHAR(50),
                days_moonphase FLOAT, 
                days_precip FLOAT, 
                days_precipcover FLOAT, 
                days_precipprob FLOAT,
                days_preciptype VARCHAR(50),
                days_pressure FLOAT, 
                days_severerisk FLOAT, 
                days_snow FLOAT, 
                days_snowdepth FLOAT, 
                days_solarenergy FLOAT,
                days_solarradiation FLOAT,
                days_source VARCHAR(50),
                days_stations VARCHAR(200),
                days_sunrise TIME,
                days_sunriseEpoch INT,
                days_sunset TIME,
                days_sunsetEpoch INT,
                days_temp FLOAT, 
                days_tempmax FLOAT,
                days_tempmin FLOAT,
                days_uvindex FLOAT,
                days_visibility FLOAT,
                days_winddir FLOAT,
                days_windgust FLOAT,
                days_windspeed FLOAT
            );
            ''',
            ''' 
            CREATE TABLE IF NOT EXISTS days_hours (
                id_days INT AUTO_INCREMENT PRIMARY KEY,
                days_hours_cloudcover FLOAT,
                days_hours_conditions VARCHAR(200),
                days_hours_datetime TIME,
                days_hours_datetimeEpoch INT, 
                days_hours_dew FLOAT,
                days_hours_feelslike FLOAT,
                days_hours_humidity FLOAT,
                days_hours_icon VARCHAR(200),
                days_hours_precip FLOAT,
                days_hours_precipprob FLOAT,
                days_hours_preciptype VARCHAR(200),
                days_hours_pressure FLOAT,
                days_hours_severerisk FLOAT,
                days_hours_snow FLOAT,
                days_hours_snowdepth FLOAT,
                days_hours_solarenergy FLOAT,
                days_hours_solarradiation FLOAT,
                days_hours_source VARCHAR(50),
                days_hours_stations VARCHAR(200),
                days_hours_temp FLOAT,
                days_hours_uvindex FLOAT,
                days_hours_visibility FLOAT,
                days_hours_winddir FLOAT,
                days_hours_windgust FLOAT,
                days_hours_windspeed FLOAT
            );
            ''',
            ''' 
            CREATE TABLE IF NOT EXISTS city (
                id_city INT AUTO_INCREMENT PRIMARY KEY,
                address VARCHAR(50),
                description VARCHAR(300),
                latitude FLOAT,
                longitude FLOAT,
                queryCost INT, 
                resolvedAddress VARCHAR(100),
                timezone VARCHAR(100),
                tzoffset FLOAT
            );
            '''
        ]
        
        for query in tablas:
            cursor.execute(query)
    
    except Exception as e:
        print(f'Error en la creación de tablas: {e}')

def main():
    
    print('Intentando conexión a MySQL.')
    
    try:
        conexion = pymysql.connect(user='root', password='Lisandrotorre478-',
                                        host= 'localhost',   
                                        database = 'proyecto_clima',
                                        port = 3306)
        print("Conexión exitosa")
    
        cursor = conexion.cursor()
        
        crear_tablas(cursor)
        
        dataframes = {
            'stations' : dfPandas_stations,
            'current_conditions' : dfPandas_currentConditions,
            'days' : dfPandas_Days,
            'days_hours' : dfPandas_DayHours,
            'city' : dfPandas_Original 
        }
        
        for tabla, df in dataframes.items():
            
            columnas = df.columns.tolist()
            datos = [tuple(row) for row in df.values]
        
            insertar_datos_a_tabla(cursor, tabla, columnas, datos)
        
        conexion.commit()
        print('Datos insertados exitosamente')
    
    except Exception as e:
        print(f'Error en la conexión: {e}')
        
    finally: 
        conexion.close()
        print('Conexión cerrada con éxito.')

main()

Intentando conexión a MySQL.
Conexión exitosa
Datos insertados correctamente en la tabla stations.
Error en la inserción de datos: "current_conditions" / (1241, 'Operand should contain 1 column(s)')
Datos insertados correctamente en la tabla days.
Error en la inserción de datos: "days_hours" / (1241, 'Operand should contain 1 column(s)')
Datos insertados correctamente en la tabla city.
Datos insertados exitosamente
Conexión cerrada con éxito.
